# Yelp API Webscraping

In [2]:
import pandas as pd 
import numpy as np
import json
import requests
import time, datetime
import math

# Compiled Code
- Don't forget to comment so we can understand!

## known issues:
- function populates dataframe with NaN rows
- do we want business name?
- businesses have multiple categories and function only picks up one at a time
    - currently the 'alias' portion picks up the alias of the business (name), not the category alias

In [3]:
# NOTHING IN THIS CELL NEEDS TO GET CHANGED 
# IF YOU HAVE YOUR API KEY IN 'creds.json' IN ./Assets

# format your json file as a dictionary containing api key with DOUBLE QUOTES
# {"api": "your_super_long_api_key"}
CREDS_FILE = open('../Assets/creds.json')

# load credentials into variable
YELP_CREDENTIALS = json.loads(CREDS_FILE.read())
API_KEY = YELP_CREDENTIALS['api']
HEADERS = {'Authorization': 'Bearer %s' % API_KEY}

# this is the url we use to make broad business searches
# https://www.yelp.com/developers/documentation/v3/business_search

URL = 'https://api.yelp.com/v3/businesses/search'

In [4]:
def get_businesses(location, save):
    
    businesses_object = []
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'radius': 3200,
            'offset': offset
        }
        response = requests.get(URL, headers=HEADERS, params=params)
        if response.status_code == 200:
            businesses_object += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break


    # empty list container containing dictionaries representing each unique business
    businesses = []
    
    # looking at each business which is passed into our function
    for i, business in enumerate(list(businesses_object)):
        
        # each dictionary will contain all of the desired information from each business
        business_dict = {}

        if 'price' in businesses_object[i].keys():
            business_dict['id']           = business['id']
            business_dict['latitude']     = business['coordinates']['latitude']
            business_dict['longitude']    = business['coordinates']['longitude']
            business_dict['price']        = business['price']
            business_dict['review_count'] = business['review_count']
            business_dict['rating']       = business['rating']
            business_dict['zip_code']     = business['location']['zip_code']
            business_dict['city']         = business['location']['city']
            business_dict['alias']        = business['alias']
           
        num_categories = len(businesses_object[i]['categories'])
        
        category_list = []
        
        for num in range(num_categories):
            category = businesses_object[i]['categories'][num]['alias']
            category_list.append(category)
            business_dict['category']     = category_list

        # populate the business list with each business dictionary
        businesses.append(business_dict)

        # just a little sanity check to see how far along our function is going
        if (i+1) % 100 == 0:
            print(f'{i+1} out of {len(businesses_object)}')
    
    businesses = pd.DataFrame(businesses)
    
    # Saves the business dataframe to a csv file with current time
    if save:
        businesses.to_csv(businesses_path,index=False)
        now = datetime.datetime.now()
        businesses_path = "../data/" + location +"_businesses" + now.strftime("%Y-%m-%d_%H_%M") + ".csv"

    return businesses

In [19]:
zips = [90001,
90002,
90003,
90004,
90005,
90006,
90007,
90008,
90009,
90010,
90011,
90012,
90013,
90014,
90015,
90016,
90017,
90018,
90019,
90020,
90021,
90022,
90023,
90024,
90025,
90026,
90027,
90028,
90029,
90030,
90031,
90032,
90033,
90034,
90035,
90036,
90037,
90038,
90039,
90040,
90041,
90042,
90043,
90044,
90045,
90046,
90047,
90048,
90049,
90050,
90051,
90052,
90053,
90054,
90055,
90056,
90057,
90058,
90059,
90060,
90061,
90062,
90063,
90064,
90065,
90066,
90067,
90068,
90069,
90070,
90071,
90072,
90073,
90074,
90075,
90076,
90077,
90078,
90079,
90080,
90081,
90082,
90083,
90084,
90086,
90087,
90088,
90089,
90090,
90091,
90093,
90094,
90095,
90096,
90099,
90134,
90189,
90230]

In [31]:
zips = [str(num) for num in zips]

In [33]:
for zipcode in zips:
    get_businesses(zipcode)

100 out of 514
200 out of 514
300 out of 514
400 out of 514
500 out of 514
100 out of 386
200 out of 386
300 out of 386
100 out of 349
200 out of 349
300 out of 349
100 out of 950
200 out of 950
300 out of 950
400 out of 950
500 out of 950
600 out of 950
700 out of 950
800 out of 950
900 out of 950
100 out of 1000
200 out of 1000
300 out of 1000
400 out of 1000
500 out of 1000
600 out of 1000
700 out of 1000
800 out of 1000
900 out of 1000
1000 out of 1000
100 out of 1000
200 out of 1000
300 out of 1000
400 out of 1000
500 out of 1000
600 out of 1000
700 out of 1000
800 out of 1000
900 out of 1000
1000 out of 1000
100 out of 1000
200 out of 1000
300 out of 1000
400 out of 1000
500 out of 1000
600 out of 1000
700 out of 1000
800 out of 1000
900 out of 1000
1000 out of 1000
100 out of 260
200 out of 260
100 out of 707
200 out of 707
300 out of 707
400 out of 707
500 out of 707
600 out of 707
700 out of 707
100 out of 1000
200 out of 1000
300 out of 1000
400 out of 1000
500 out of 1000
60

100 out of 950
200 out of 950
300 out of 950
400 out of 950
500 out of 950
600 out of 950
700 out of 950
800 out of 950
900 out of 950
100 out of 950
200 out of 950
300 out of 950
400 out of 950
500 out of 950
600 out of 950
700 out of 950
800 out of 950
900 out of 950
100 out of 1000
200 out of 1000
300 out of 1000
400 out of 1000
500 out of 1000
600 out of 1000
700 out of 1000
800 out of 1000
900 out of 1000
1000 out of 1000
100 out of 736
200 out of 736
300 out of 736
400 out of 736
500 out of 736
600 out of 736
700 out of 736
100 out of 950
200 out of 950
300 out of 950
400 out of 950
500 out of 950
600 out of 950
700 out of 950
800 out of 950
900 out of 950
100 out of 590
200 out of 590
300 out of 590
400 out of 590
500 out of 590
100 out of 703
200 out of 703
300 out of 703
400 out of 703
500 out of 703
600 out of 703
700 out of 703
100 out of 1000
200 out of 1000
300 out of 1000
400 out of 1000
500 out of 1000
600 out of 1000
700 out of 1000
800 out of 1000
900 out of 1000
1000 

## aerika section

## echo section

added comment

In [16]:
import time

In [17]:
# stole this function from project 3 - jerry-----Echo: comment out a few lines to make them work with time in next cell

def get_price(yelp_object):
    businesses = []
    # empty list container containing dictionaries representing each unique business
    
    businesses_object = yelp_object['businesses']
    
    # looking at each subreddit which is passed into our function
    for i, business in enumerate(list(businesses_object)):
        
        # each dictionary will contain all of the desired information from each post
        business_dict = {}

        if 'price' in businesses_object[i].keys():
            business_dict['id']           = business['id']
            business_dict['latitude']     = business['coordinates']['latitude']
            business_dict['longitude']    = business['coordinates']['longitude']
            business_dict['price']        = business['price']
            business_dict['review_count'] = business['review_count']
            business_dict['rating']       = business['rating']
            business_dict['zip_code']     = business['location']['zip_code']
            business_dict['city']         = business['location']['city']
            business_dict['alias']        = business['alias']

        # populate the posts list with each post dictionary
        businesses.append(business_dict)

        # just a little sanity check to see how far along our function is going
        #print(f'{i+1} out of {len(businesses_object)}')
    return businesses
    #businesses = pd.DataFrame(businesses)
   

In [ ]:
##let the function run 1 time every second to get data list

In [18]:
business_list = []  ## set n to your ideal number. Yelp claims we canget 5000 a day
n=0
while n<6:
    business_list.extend(get_price(yelp))
    time.sleep(1)
    n+=1

In [22]:
df = pd.DataFrame(business_list)

In [24]:
df

,id,latitude,longitude,price,review_count,rating,zip_code,city,alias
0,CcqraT0cuGKYEcZ1ri_kxg,34.040403,-118.253512,$$,689,5.0,90015,Los Angeles,broken-mouth-lees-homestyle-los-angeles-5
1,KQBGm5G8IDkE8LeNY45mbA,34.045605,-118.236061,$$,8205,4.0,90013,Los Angeles,wurstküche-los-angeles-2
2,pjh40JY5YwWeV8aKhkXERg,34.065770,-118.308470,$,702,4.5,90020,Los Angeles,myungrang-hot-dog-california-market-la-los-ang...
3,b4SH4SbQUJfXxh6hNkF0wg,34.050529,-118.248619,$,5597,4.0,90013,Los Angeles,eggslut-los-angeles-7
4,bvpxd7o3RZtz50YLpDsgLA,34.057125,-118.346074,$,1266,4.5,90019,Los Angeles,el-chato-taco-truck-los-angeles-2
...,...,...,...,...,...,...,...,...,...
115,oNHjZ3wC0F9l5KpuqHd45w,34.114824,-118.269945,$,1160,4.5,90027,Los Angeles,rickys-fish-tacos-los-angeles-3
116,23Qrcz2i9e2e8MstUIIO0A,34.062181,-118.348153,$$,1527,4.5,90036,Los Angeles,yuko-kitchen-los-angeles
117,HXWdcnzYG1zmf0vDplmEEQ,34.059960,-118.419820,$$,354,4.0,90067,Los Angeles,the-crack-shack-los-angeles-2
118,F1_EZV0z5gjoZu6K4BUUzQ,34.154819,-118.431921,$$,314,4.5,91423,Sherman Oaks,hot-motha-clucker-sherman-oaks


In [ ]:
#df.to_csv('../data/echo1023.csv')

In [30]:
df = pd.read_csv('../data/echo1023.csv',index_col=0)
df

,id,latitude,longitude,price,review_count,rating,zip_code,city,alias
0,CcqraT0cuGKYEcZ1ri_kxg,34.040403,-118.253512,$$,689.0,5.0,90015.0,Los Angeles,broken-mouth-lees-homestyle-los-angeles-5
1,KQBGm5G8IDkE8LeNY45mbA,34.045605,-118.236061,$$,8205.0,4.0,90013.0,Los Angeles,wurstküche-los-angeles-2
2,pjh40JY5YwWeV8aKhkXERg,34.065770,-118.308470,$,702.0,4.5,90020.0,Los Angeles,myungrang-hot-dog-california-market-la-los-ang...
3,b4SH4SbQUJfXxh6hNkF0wg,34.050529,-118.248619,$,5597.0,4.0,90013.0,Los Angeles,eggslut-los-angeles-7
4,sYn3SNQP-j2t2XSwjlCbRg,34.064690,-118.308760,$$,1618.0,4.5,90020.0,Los Angeles,montys-good-burger-los-angeles
...,...,...,...,...,...,...,...,...,...
6095,CbW8U0QAwh5XRkaLt0xNZA,34.077585,-118.259599,$$,4023.0,4.0,90026.0,Los Angeles,masa-of-echo-park-los-angeles
6096,fjWpZPr4YDJRKdnj0fqlkg,34.083320,-118.327380,$$,33.0,5.0,90038.0,Los Angeles,omakase-teriyaki-los-angeles
6097,ohosmz6FXVAeoW5nUkYwng,34.090596,-118.277172,$$,2151.0,4.5,90026.0,Los Angeles,pine-and-crane-los-angeles
6098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df[df.isna().any(axis=1)] ## show all the rows with nan

,id,latitude,longitude,price,review_count,rating,zip_code,city,alias
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## gonna drop the nans before we learning the missing values

In [35]:
df = df.dropna()

,id,latitude,longitude,price,review_count,rating,zip_code,city,alias
0,CcqraT0cuGKYEcZ1ri_kxg,34.040403,-118.253512,$$,689.0,5.0,90015.0,Los Angeles,broken-mouth-lees-homestyle-los-angeles-5
1,KQBGm5G8IDkE8LeNY45mbA,34.045605,-118.236061,$$,8205.0,4.0,90013.0,Los Angeles,wurstküche-los-angeles-2
2,pjh40JY5YwWeV8aKhkXERg,34.065770,-118.308470,$,702.0,4.5,90020.0,Los Angeles,myungrang-hot-dog-california-market-la-los-ang...
3,b4SH4SbQUJfXxh6hNkF0wg,34.050529,-118.248619,$,5597.0,4.0,90013.0,Los Angeles,eggslut-los-angeles-7
4,sYn3SNQP-j2t2XSwjlCbRg,34.064690,-118.308760,$$,1618.0,4.5,90020.0,Los Angeles,montys-good-burger-los-angeles
...,...,...,...,...,...,...,...,...,...
6093,WjuO8PhtqINeWqm-ei8NQA,34.069300,-118.292530,$$,508.0,4.5,90020.0,Los Angeles,dumpling-house-los-angeles-2
6095,CbW8U0QAwh5XRkaLt0xNZA,34.077585,-118.259599,$$,4023.0,4.0,90026.0,Los Angeles,masa-of-echo-park-los-angeles
6096,fjWpZPr4YDJRKdnj0fqlkg,34.083320,-118.327380,$$,33.0,5.0,90038.0,Los Angeles,omakase-teriyaki-los-angeles
6097,ohosmz6FXVAeoW5nUkYwng,34.090596,-118.277172,$$,2151.0,4.5,90026.0,Los Angeles,pine-and-crane-los-angeles


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6100 entries, 0 to 6099
Data columns (total 9 columns):
id              5490 non-null object
latitude        5490 non-null float64
longitude       5490 non-null float64
price           5490 non-null object
review_count    5490 non-null float64
rating          5490 non-null float64
zip_code        5490 non-null float64
city            5490 non-null object
alias           5490 non-null object
dtypes: float64(5), object(4)
memory usage: 476.6+ KB


In [ ]:
## seems there is no business with only a few columns of nan, dont know what to clean yet

## jerry section

In [5]:
# https://en.wikipedia.org/wiki/Earth_radius

def get_earth_radius(lat):
    '''
    Calculates radius of Earth at given latitude, assuming oblate spheroid geometry.
    
    Arguments:
        lat (float):        latitude, in decimal degrees
        
    Returns:
        radius (float) in meters
    '''
    
    lat = math.radians(lat)
    
    # geocentric radius formula
    a = 6_378_137               # equatorial radius
    b = 6_356_752.3             # polar radius
    numer = (a**2 * math.cos(lat))**2 + (b**2 * math.sin(lat))**2
    denom = (a * math.cos(lat))**2 + (b * math.sin(lat))**2
    
    return math.sqrt(numer / denom) 

In [6]:
def get_new_point(lat1, lon1, distance, bearing):
    '''
    Generates destination geographic coordinate based on starting point, 
    spherical distance traveled, and bearing.
    Use only for short distances (< 100_000).
    
    Arguments:
        lat1 (float):       starting latitude, in decimal degrees
        lon1 (float):       starting longitude, in decimal degrees
        distance (float):   distance to closest point, in meters
        bearing (float):    direction from starting point, in decimal degrees
        
    Returns:
        tuple containing new coordinates
        
    Dependencies:
        get_earth_radius()
    '''
    
    er = get_earth_radius(lat1)
    
    # converting to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    
    # formulae taken from https://www.movable-type.co.uk/scripts/latlong.html
    lat2 = math.asin(
            math.sin(lat1) * math.cos(distance / er)
            + math.cos(lat1) * math.sin(distance / er) * math.cos(bearing))
    
    lon2 = lon1 + math.atan2(
            math.sin(bearing) * math.sin(distance / er) * math.cos(lat1),
            math.cos(distance / er) - math.sin(lat1) * math.sin(lat2))
    
    return math.degrees(lat2), math.degrees(lon2)

In [7]:
# picking seven smaller circles to "equal" one big circle
# https://en.wikipedia.org/wiki/Haversine_formula
# https://stackoverflow.com/questions/639695/
# https://stackoverflow.com/questions/7222382/
# https://www2.stetson.edu/~efriedma/circovcir/

def get_six_points(lat1, lon1, distance, rotation=0):
    '''
    Generates 6 additional geographic coordinates hexagonally arranged some distance from starting point.
    
    Arguments:
        lat1 (float):       starting latitude, in decimal degrees
        lon1 (float):       starting longitude, in decimal degrees
        distance (float):   distance to closest point, in meters
        rotation (float):   rotational offset, in degrees (recommended range -30 to 30)
        
    Returns:
        list of dictionaries with {'latitude': lat2, 'longitude': lon2}, excluding starting point
        
    Dependencies:
        get_new_point()
        get_earth_radius()
    '''    
    points = []
    
    # six directions
    bearings = [math.radians(deg) for deg in range(rotation, 360, 60)]

    for bearing in bearings:
        point = {}
        lat2, lon2 = get_new_point(lat1, lon1, distance, bearing)
        point['latitude'] = lat2
        point['longitude'] = lon2
        points.append(point)

    return points

In [140]:
get_six_points(34.05219, -118.2437, 2000)

[{'latitude': 34.070175103661924, 'longitude': -118.2437},
 {'latitude': 34.06118112078147, 'longitude': -118.22489893181573},
 {'latitude': 34.04319601764459, 'longitude': -118.22490291992644},
 {'latitude': 34.034204896338075, 'longitude': -118.2437},
 {'latitude': 34.04319601764459, 'longitude': -118.26249708007357},
 {'latitude': 34.06118112078147, 'longitude': -118.26250106818426}]

In [8]:
def get_eighteen_points(lat1, lon1, distance, rotation=0):
    '''
    Generates 18 additional geographic coordinates hexagonally arranged some distance from starting point.
    
    Arguments:
        lat1 (float):       starting latitude, in decimal degrees
        lon1 (float):       starting longitude, in decimal degrees
        distance (float):   distance to closest point, in meters
        rotation (float):   rotational offset, in degrees (recommended range -30 to 30)
        
    Returns:
        list of dictionaries with {'latitude': lat2, 'longitude': lon2}, excluding starting point
        
    Dependencies:
        get_six_points()
        get_new_point()
        get_earth_radius()
    '''    
    points = []
    
    # distance between centers of circles
    distance2 = distance * 2               # along six "axes" of inner hexagon
    distance3 = math.sqrt(3) * distance     # between six "axes" of inner hexagon

    # inner hexagon, outer hexagon, in-between hexagon
    points.extend(get_six_points(lat1, lon1, distance, rotation))
    points.extend(get_six_points(lat1, lon1, distance2, rotation))
    points.extend(get_six_points(lat1, lon1, distance3, rotation + 30))

    return points

In [145]:
get_eighteen_points(34.05219, -118.2437, 2000)

[{'latitude': 34.070175103661924, 'longitude': -118.2437},
 {'latitude': 34.06118112078147, 'longitude': -118.22489893181573},
 {'latitude': 34.04319601764459, 'longitude': -118.22490291992644},
 {'latitude': 34.034204896338075, 'longitude': -118.2437},
 {'latitude': 34.04319601764459, 'longitude': -118.26249708007357},
 {'latitude': 34.06118112078147, 'longitude': -118.26250106818426},
 {'latitude': 34.08816020732387, 'longitude': -118.2437},
 {'latitude': 34.07016937841362, 'longitude': -118.20609387459538},
 {'latitude': 34.03419917529014, 'longitude': -118.20610982703664},
 {'latitude': 34.016219792676154, 'longitude': -118.2437},
 {'latitude': 34.03419917529014, 'longitude': -118.28129017296335},
 {'latitude': 34.07016937841362, 'longitude': -118.28130612540461},
 {'latitude': 34.07916622391802, 'longitude': -118.22489494077674},
 {'latitude': 34.05218427685251, 'longitude': -118.206101853743},
 {'latitude': 34.02521091450738, 'longitude': -118.22490690511145},
 {'latitude': 34.02

<img src="../assets/surrounding_points.jpg" width="600" alt="http://www.copypastemap.com/" title="http://www.copypastemap.com/">

In [9]:
def add_businesses(response, business_list):
    '''
    Adds businesses to a list container.
    
    Arguments:
        response:           the API request object
        business_list:      an empty (or existing) list container
        
    Returns:
        the modified list container
    '''    
      
    bus_obj = response.json()['businesses']

    for i, business in enumerate(list(bus_obj)):

    # each dictionary will contain all of the desired information from each business
        business_dict = {}

        if 'price' in bus_obj[i].keys():
            business_dict['id']           = business['id']
            business_dict['latitude']     = business['coordinates']['latitude']
            business_dict['longitude']    = business['coordinates']['longitude']
            business_dict['price']        = business['price']
            business_dict['review_count'] = business['review_count']
            business_dict['rating']       = business['rating']
            business_dict['zip_code']     = business['location']['zip_code']
            business_dict['city']         = business['location']['city']
            business_dict['alias']        = business['alias']
            business_dict['categories']   = business['categories']

            business_list.append(business_dict)
        
    return business_list

In [10]:
def get_coordinates(zipcode):
    '''
    Converts a five-digit USPS zip code to geographic coordinates.
    
    Arguments:
        zipcode:            a five-digit string
        
    Returns:
        a tuple containing (latitude, longitude), in decimal degrees
    ''' 
    zipcoords = json.loads(open('../Assets/zip_code_coordinates.json').read())
    if len(zipcode) == 5 and zipcode.isnumeric():
        latitude = zipcoords[zipcode]['latitude']
        longitude = zipcoords[zipcode]['longitude']
    return latitude, longitude

In [11]:
def get_businesses(lat, lon, radius, force=False):
    '''
    Grabs as many businesses as allowed by the Yelp API given geographic coordinates and search radius.
    
    Arguments:
        lat:            latitude, in decimal degrees
        lon:            longitude, in decimal degrees
        radius:         search radius, in meters
        force:          forces the function to gather results even if over 1000 businesses are returned
        
    Returns:
        a list containing dictionaries of each business listing
    
    Dependencies:
        add_businesses()
    '''    
    radius = int(radius)
    
    businesses = []
    
    params = {
        'limit': 50, 
        'latitude': lat,
        'longitude': lon,
        'radius': radius,
        'offset': 0,
        'sort_by': 'distance'
    }

    response = requests.get(URL, headers=HEADERS, params=params)
    
    total = response.json()['total']
    
    if (total > 1000):
        print(f'caution: query returns approximately {round(total, -2)} listings.')
        
    if force == False:
        return None
    
    else:
        add_businesses(response, businesses)
        
        maximum = min(total, 1000)
        
        for offset in range(50, maximum, 50):
            params['offset'] = offset
            response = requests.get(URL, headers=HEADERS, params=params)

            if response.status_code == 500:
                time.sleep(2)
                response = requests.get(URL, headers=HEADERS, params=params)

            add_businesses(response, businesses)

            if offset % 100 == 0:
                print(f'{offset} out of {total}')
                
            time.sleep(.2)
            
        print(f'finished! {maximum} out of {total}')
        return businesses

In [12]:
# https://stackoverflow.com/questions/9427163/

def remove_duplicates(list_of_dictionaries):
    '''
    Removes duplicate dictionaries in the businesses list by matching business ID.
    
    Arguments:
        list_of_dictionaries:       the list containing business dictionaries
        
    Returns:
        a modified list without duplicate entries
    '''    
    
    seen = set()
    new_list = []
    
    for dictionary in list_of_dictionaries:
        i = dictionary['id']
        if i not in seen:
            seen.add(i)
            new_list.append(dictionary)
    print(f'{len(new_list)} unique listings from {len(list_of_dictionaries)} aggregated results')
    return new_list

In [13]:
def shotgun(lat, lon, radius, spray=6, rotation=0):
    '''
    Grabs businesses using get_businesses(), but aggregates search queries with
    smaller radii to circumvent 1000-business limit imposed by API.
    
    Arguments:
        lat:            latitude, in decimal degrees
        lon:            longitude, in decimal degrees
        radius:         search radius to emulate, in meters
        spray:          parameter to use get_six_points() or get_eighteen_points()
        rotation:       parameter to rotate the geographic coordinates, in degrees
        
    Returns:
        a list containing dictionaries of each business listing
    
    Dependencies:
        get_businesses()
        add_businesses()
        get_six_points()
        get_eighteen_points()
        get_new_point()
        get_earth_radius()
        remove_duplicates()
    '''        
    start = time.time()
    businesses = []
    
    points = [{'latitude': lat, 'longitude': lon}]
    
    if spray == 6:
        distance = radius * math.sqrt(3) / 2
        points.extend(get_six_points(lat, lon, distance, rotation))
    elif spray == 18:
        distance = radius * math.sqrt(3) / math.sqrt(13)
        points.extend(get_eighteen_points(lat, lon, distance, rotation))
        
    small_radius = distance / math.sqrt(3)
    
    if get_businesses(lat, lon, radius) == None:
        print('attempting shotgun')
        
        for i, point in enumerate(points):
            print(f'=========== {i+1} of {len(points)} ============')
            pellet = get_businesses(point['latitude'], point['longitude'], small_radius, force=True)
            businesses.extend(pellet)
            time.sleep(.2)
        
        cleaned = remove_duplicates(businesses)
        
        end = time.time()
        elapsed = end - start
        
        print(f'elapsed time: {round(elapsed / 60, 1)} min.')
        print(f'overall efficiency: {round(elapsed / len(cleaned) * 1000, 1)} seconds per 1000 listings.')
        return cleaned
    
    else:
        print('query does not exceed 1000 listings. please use get_businesses() instead.')
        return None

In [99]:
lat, lon = get_coordinates('90210')
temp_list_2 = shotgun(lat, lon, 500, spray=18)

In [104]:
temp_list_2[:5]

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750


755